# **Filogenia**

A filogenia é um campo que estuda as relações evolutivas entre organismos. Existem várias técnicas e métodos para a construção de árvores filogenéticas, e a escolha dependerá do tipo de dados disponíveis e das perguntas a responder.

Para análise filogenética utilizando o algoritmo **Needleman-Wunsch** devemos garantir estas etapas:

- **Loop de Alinhamento**:

    *Este loop deve invocar o algoritmo ***Needleman-Wunsch*** para cada par de sequências e, em seguida, atualizar a matriz ou estrutura de dados que armazena as pontuações ou distâncias entre as sequências.*

- **Matriz com um Dado Score**:

    *A matriz pode ser inicializada com zeros, e o loop de alinhamento será responsável por preenchê-la com os valores de pontuação calculados pelo algoritmo ***Needleman-Wunsch.***

- **Cálculo e Agrupamento Progressivo**:

    *Após realizar os alinhamentos progressivos e preencher a matriz de distâncias, é necessário implementar um método para calcular as distâncias progressivas e agrupar as sequências de acordo com essas distâncias. ***nw_ciclico***

- **Profit**:

    *O "Profit" refere-se à avaliação da qualidade do agrupamento ou à construção da árvore filogenética. Isso pode envolver a análise da estabilidade do agrupamento, a avaliação do bootstrap, ou a construção da árvore final.*

Para iniciar a **análise filogenética** necessitamos de fazer algumas modificações aos algoritmos desenvolvidos previamente apenas para o alinhamento:

In [121]:
def needleman_wunsch(seq1, seq2, score_subst, score_space):
    """
    Modificações/alterações feitas ao algoritmo base:
    - Adição de Pontuações de Espaço: As pontuações de espaço agora são aplicadas durante a inicialização da primeira coluna e linha da matriz de pontuação.
    - Aprimoramento no Traceback: Durante o traceback, o código agora verifica se o caminho escolhido é uma correspondência (D), exclusão (U), ou inserção (L). Isso é útil para gerar corretamente as sequências alinhadas.
    - Escolha da Melhor Opção: O código agora seleciona a melhor opção entre correspondência, exclusão e inserção, usando a função max e atribuindo best_choice à posição atual na matriz de pontuação.
    - Scores.
    """
    len_1, len_2 = len(seq1), len(seq2)
    
    # Inicializar a matriz de pontuação e a matriz de traceback
    score = [[0] * (len_2 + 1) for _ in range(len_1 + 1)]
    traceback = [[''] * (len_2 + 1) for _ in range(len_1 + 1)]
    
    # Inicializar a primeira coluna e linha com penalidades de espaço
    for i in range(len_1 + 1):
        score[i][0]     = i * score_space
        traceback[i][0] = 'U'
    for j in range(len_2 + 1):
        score[0][j]     = j * score_space
        traceback[0][j] = 'L'

    # Preencher a matriz de pontuação e a matriz de traceback
    for i in range(1, len_1 + 1):
        for j in range(1, len_2 + 1):
            match  = score[i-1][j-1] + (score_subst if seq1[i-1] == seq2[j-1] else -1)
            delete = score[i-1][j] + score_space
            insert = score[i][j-1] + score_space

            choices = [match, delete, insert]
            best_choice = max(choices)

            score[i][j] = best_choice
            traceback[i][j] = 'D' if best_choice == delete else ('U' if best_choice == insert else 'L')

    # Realizar o traceback para obter as sequências alinhadas
    aligned_seq1, aligned_seq2 = '', ''
    i, j = len_1, len_2
    while i > 0 or j > 0:
        if traceback[i][j] == 'D':
            aligned_seq1 = seq1[i-1] + aligned_seq1
            aligned_seq2 = seq2[j-1] + aligned_seq2
            i -= 1
            j -= 1
        elif traceback[i][j] == 'U':
            aligned_seq1 = seq1[i-1] + aligned_seq1
            aligned_seq2 = '-' + aligned_seq2
            i -= 1
        elif traceback[i][j] == 'L':
            aligned_seq1 = '-' + aligned_seq1
            aligned_seq2 = seq2[j-1] + aligned_seq2
            j -= 1

    return aligned_seq1, aligned_seq2, score[len_1][len_2]

# Adiciona espaços (lacunas) ao final da sequência para atingir o comprimento desejado.
def aprimorar_seq(string, length):
    """
    Modificações:
    - Adição de Parâmetro de Comprimento (length): Agora a função aceita um comprimento desejado como parâmetro, garantindo que a sequência seja estendida até o comprimento especificado.
    """
    return string + '-' * (length - len(string))

# Imprime a matriz de pontuação.
def print_matrix(string_1, string_2, matrix):
    """
    Modificações:
    - Adição de Rótulos de Sequência: As sequências string_1 e string_2 são agora usadas para rotular as linhas e colunas da matriz.
    """
    print(f'    {string_2}')
    for i, row in enumerate(matrix):
        row_str = ' '.join(map(str, row))
        print(f'{string_1[i]} {row_str}')


Após adaptar os algoritmos dos alinhamentos progressivos e matriz de distâncias, é necessário implementar um método para calcular as distâncias progressivas e agrupar as sequências de acordo com essas distâncias: **nw_ciclico**

In [122]:
# A função tem como objetivo calcular a dissimilaridade entre todas as sequências em sequencias usando o algoritmo de Needleman-Wunsch
def nw_ciclico(sequencias, score_subst, score_space):
    """
    A função itera sobre todas as combinações únicas de pares de sequências, alinha cada par usando o algoritmo de Needleman-Wunsch e calcula a dissimilaridade com base no escore obtido.
    A dissimilaridade é calculada por 1-(score/max(len(aligned_seq_i), len(aligned_seq_j)).
    A matriz de dissimilaridade resultante (distancia_matriz) é simétrica.
    """
    num_sequencias = len(sequencias) # Obtém o número total de sequências.
    distancia_matriz = [[0.0] * num_sequencias for _ in range(num_sequencias)] # Inicializa uma matriz de dissimilaridade com dimensões num_sequencias x num_sequencias, preenchida com zeros.

    for i in range(num_sequencias): # Os loops aninhados for i in range(num_sequencias) e for j in range(i + 1, num_sequencias) garantem que cada par de sequências seja considerado apenas uma vez.
        for j in range(i + 1, num_sequencias):
            nome_i, seq_i = sequencias[i]
            nome_j, seq_j = sequencias[j]

            # Para cada par de sequências, needleman_wunsch é chamado para realizar o alinhamento global de Needleman-Wunsch.
            aligned_seq_i, aligned_seq_j, score = needleman_wunsch(seq_i, seq_j, score_subst, score_space)

            # A dissimilaridade é calculada usando o escore normalizado pela diferença no comprimento das sequências alinhadas.
            dissimilaridade        = 1.0 - (score / max(len(aligned_seq_i), len(aligned_seq_j)))
            distancia_matriz[i][j] = dissimilaridade
            distancia_matriz[j][i] = dissimilaridade

    return distancia_matriz


In [127]:
# Imports para criar a árvore filogenética com biopython
from Bio import pairwise2
from Bio import Phylo
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment
from Bio.Phylo.TreeConstruction import DistanceCalculator, DistanceTreeConstructor
from io import StringIO

# Este código utiliza a biblioteca Biopython para realizar cálculos e construção da árvore filogenética com base nas sequências fornecidas.
def criar_arvore_filogenetica(distancia_matriz, sequencias):
    """
    A lista sequencias contém tuplas com o nome e a sequência de cada entidade. Os nomes são extraídos para uso posterior.
    É criada uma instância da classe MultipleSeqAlignment usando as sequências ajustadas ao mesmo comprimento. Isso é feito para garantir que todas as sequências tenham o mesmo comprimento antes de calcular a matriz de distância.
    A matriz de distância é calculada usando a identidade como métrica. A identidade é uma métrica comum na construção de árvores filogenéticas e representa a fração de locais alinhados idênticos entre as sequências.
    A classe DistanceTreeConstructor é usada com o método UPGMA (Unweighted Pair Group Method with Arithmetic mean) para construir a árvore filogenética. Este método é adequado para dados de sequência molecular.
    A árvore filogenética é impressa usando a função print(tree).
    """
    # Obter os nomes e sequências das entidades
    nomes = [nome for nome, _ in sequencias]

    # Criar uma instância de MultipleSeqAlignment com as sequências ajustadas ao mesmo comprimento
    msa = MultipleSeqAlignment([SeqRecord(Seq(aprimorar_seq(seq, len(max(sequencias, key=lambda x: len(x[1]))[1]))), id=nome) for nome, seq in sequencias])

    # Calcular a matriz de distância usando a identidade como métrica
    calculator = DistanceCalculator('identity')
    dm         = calculator.get_distance(msa)

    # Construir a árvore filogenética usando o método UPGMA
    constructor = DistanceTreeConstructor(calculator)
    tree        = constructor.upgma(dm)

    # Imprimir a árvore filogenética
    print(tree)


A partir dos algoritmos modificados, nw_ciclico, e uma função para "desenhar" a árvore a partir da distância da matriz podemos criar dois tipo de árvores filogenéticas em python.

In [148]:
# Exemplo de uso
s1 = 'ATCTCGGCTAAAC'
s2 = 'ACCGTATCTCGGTGCA'

# Lista de tuplas contendo nome e sequência, alterar estes parâmetros para o número de sequências a analisar
sequencias = [('s1', s1), ('s2', s2)]

# Parâmetros
score_subst = 2
score_space = -4

# Aplicar Needleman-Wunsch ciclico
distancia_matriz = nw_ciclico(sequencias, score_subst, score_space)

# Cria a 1ª árvore filogenética 
print("1ª árvore:")
criar_arvore_filogenetica(distancia_matriz, sequencias)

# Cria a 2ª árvore filogenética usando outro método
print("2ª árvore:")
tree_str = "((s2:0.375, s1:0.375)Inner1:0.0)RootedTree;"
tree = Phylo.read(StringIO(tree_str), "newick")

# Exibir a árvore em formato de texto
Phylo.draw_ascii(tree)


1ª árvore:
Tree(rooted=True)
    Clade(branch_length=0, name='Inner1')
        Clade(branch_length=0.375, name='s2')
        Clade(branch_length=0.375, name='s1')
2ª árvore:
  ___________________________________________________________________________ s2
_|
 |___________________________________________________________________________ s1

